In [1]:
import collections
import math
import torch
from torch import nn
from d2l import torch as d2l

In [26]:
class Seq2SeqEncoder(d2l.Encoder):
    # 用于序列到序列学习的循环神经网络编码器
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, dropout=0, **kwargs):
        super(Seq2SeqEncoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers, dropout=dropout)

    def forward(self, X, *args):
        X = self.embedding(X)
        X = X.permute(1, 0, 2)
        output, state = self.rnn(X)
        # output的形状:(num_steps,batch_size,num_hiddens)
        # state的形状:(num_layers,batch_size,num_hiddens)
        return output, state

# #@save
# class Seq2SeqEncoder(d2l.Encoder):
#     """用于序列到序列学习的循环神经网络编码器"""
#     def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
#                  dropout=0, **kwargs):
#         super(Seq2SeqEncoder, self).__init__(**kwargs)
#         # 嵌入层
#         self.embedding = nn.Embedding(vocab_size, embed_size)
#         self.rnn = nn.GRU(embed_size, num_hiddens, num_layers,
#                           dropout=dropout)
#
#     def forward(self, X, *args):
#         # 输出'X'的形状：(batch_size,num_steps,embed_size)
#         X = self.embedding(X)
#         # 在循环神经网络模型中，第一个轴对应于时间步
#         X = X.permute(1, 0, 2)
#         # 如果未提及状态，则默认为0
#         output, state = self.rnn(X)
#         # output的形状:(num_steps,batch_size,num_hiddens)
#         # state的形状:(num_layers,batch_size,num_hiddens)
#         return output, state

In [27]:
encoder = Seq2SeqEncoder(vocab_size=10, embed_size=8, num_hiddens=16, num_layers=2)
encoder.eval()
X = torch.zeros((4, 7), dtype=torch.long)
output, state = encoder(X)

In [28]:
output.shape

torch.Size([7, 4, 16])

In [29]:
state.shape

torch.Size([2, 4, 16])

In [30]:
class Seq2SeqDecoder(d2l.Decoder):
    """用于序列到序列学习的循环神经网络解码器"""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqDecoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size + num_hiddens, num_hiddens, num_layers,
                          dropout=dropout)
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, *args):
        return enc_outputs[1]

    def forward(self, X, state):
        # 输出'X'的形状：(batch_size,num_steps,embed_size)
        X = self.embedding(X).permute(1, 0, 2)
        # 广播context，使其具有与X相同的num_steps
        context = state[-1].repeat(X.shape[0], 1, 1)
        X_and_context = torch.cat((X, context), 2)
        output, state = self.rnn(X_and_context, state)
        output = self.dense(output).permute(1, 0, 2)
        # output的形状:(batch_size,num_steps,vocab_size)
        # state的形状:(num_layers,batch_size,num_hiddens)
        return output, state

In [31]:
decoder = Seq2SeqDecoder(vocab_size=10, embed_size=8, num_hiddens=16,
                         num_layers=2)
decoder.eval()
state = decoder.init_state(encoder(X))
output, state = decoder(X, state)
output.shape, state.shape

(torch.Size([4, 7, 10]), torch.Size([2, 4, 16]))

In [1]:
import numpy as np
data = np.array([0,0,0,7,7]*300).astype(np.uint8)

In [2]:
data.shape

(1500,)